In [1]:
from bs4 import BeautifulSoup
from pyspark.sql.functions import udf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

/home/yogender/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [2]:
from bs4 import BeautifulSoup

In [199]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def soupify(rdd):
    ps = BeautifulSoup(rdd, "html.parser")
    #soup
    print(ps.prettify())
    return ps

In [176]:
text = """
<div id="theworldsgreatest" class='my-right my-hide-small my-wide toptext' style="font-family:'Segoe UI',Arial,sans-serif">
    THE WORLD'S LARGEST WEB DEVELOPER SITE
    <h1 style="font-size:300%;">THE WORLD'S LARGEST WEB DEVELOPER SITE</h1>
    <p style="font-size:160%;">Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum..</p>
</div>
</div>"""

In [177]:
data = spark.createDataFrame([[text]]).toDF("text")

In [178]:
schema = StructType([
StructField("text", StringType (),True)
])

In [179]:
data = spark.createDataFrame([[text]],schema=schema )

In [255]:
data

DataFrame[text: string]

In [180]:
#df = data.selectExpr("_1 as text")

In [181]:
#data.show(truncate=False)

In [182]:
#data.printSchema()

In [183]:
df = sc.parallelize( (["<div><p>foo</p> <p>bar</p></div>"]))

In [254]:
df.show()

+---+-----+------+
| id|title|  desc|
+---+-----+------+
|  1|  foo|foobar|
|  2|  bar|  null|
+---+-----+------+



In [204]:
convertUDF = udf(lambda z: soupify(z), StringType())

In [205]:
spark.udf.register("convertUDF ", convertUDF )

<function __main__.<lambda>(z)>

In [206]:
df2 = data.withColumn( 'soup', convertUDF(data['text']))

In [207]:
df2.show()

Py4JJavaError: An error occurred while calling o1083.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 47.0 failed 1 times, most recent failure: Lost task 0.0 in stage 47.0 (TID 47) (10.0.207.250 executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for bs4.element.NavigableString)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:773)
	at net.razorvine.pickle.Unpickler.load_newobj(Unpickler.java:777)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:299)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:123)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:136)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.

## 

In [187]:
#df2 = fd.withColumn( 'soup',convertUDF(fd['desc']))

In [188]:
df2.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/yogender/.local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/yogender/.local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/yogender/.local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/yogender/.local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/yogender/.local/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/home/yo

In [124]:
fd = sc.parallelize([(1, "foo", "<div><p>foo</p> <p>bar</p></div>", ), \
                (2, "bar", '<span><p>foo</p> <p>bar</p></span>', )]).toDF(["id", "title", "desc"]) \
.select('id', 'title', 'desc')

In [109]:
fd.show(truncate = False)

+---+-----+--------------------------------+
|id |title|desc                            |
+---+-----+--------------------------------+
|1  |foo  |<div><p>foo</p> <p>bar</p></div>|
|2  |bar  |null                            |
+---+-----+--------------------------------+



In [126]:
fd['desc']

Column<'desc'>

In [221]:
from pyspark.sql.functions import udf
from pyspark.sql import Column
from typing import Union

def parse_html(col: str):
    def parse_html_(desc: Union[None, str]):
        if desc is not None:
            ps = BeautifulSoup(desc, "lxml").find_all('p')
            return "".join(p.string for p in ps)
    return udf(parse_html_)(col)

df3 = sc.parallelize([(1, "foo", "<div><p>foo</p> <p>bar</p></div>", ), (2, "bar", None, )]).toDF(["id", "title", "desc"])


In [222]:
df3.show()

+---+-----+--------------------+
| id|title|                desc|
+---+-----+--------------------+
|  1|  foo|<div><p>foo</p> <...|
|  2|  bar|                null|
+---+-----+--------------------+



In [223]:
convertUDF2 = udf(lambda z: parse_html(z), StringType())

In [224]:
spark.udf.register("convertUDF2 ", convertUDF2 )

<function __main__.<lambda>(z)>

In [225]:
df2 = data.withColumn( 'parse_html', parse_html(df3['desc']))

AnalysisException: Resolved attribute(s) desc#608 missing from text#446 in operator !Project [text#446, parse_html_(desc#608) AS parse_html#626].;
!Project [text#446, parse_html_(desc#608) AS parse_html#626]
+- LogicalRDD [text#446], false


In [128]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

l = [(2, 1), (1,1)]
df = spark.createDataFrame(l)
df.show()

+---+---+
| _1| _2|
+---+---+
|  2|  1|
|  1|  1|
+---+---+



In [174]:

def calc_dif(x,y):
    # using the udf the calc_dif is called for every row in the dataframe
    # x and y are the values of the two columns 
    if (x>y) and (x==1):
        return x-y

udf_calc = udf(calc_dif, IntegerType())

dfNew = df.withColumn("calc", udf_calc("_1", "_2"))
dfNew.show()

+---+---+----+
| _1| _2|calc|
+---+---+----+
|  2|  1|null|
|  1|  1|null|
+---+---+----+



In [351]:
tag_soup = '''\
   <meta http-equiv="Content-Type"
       content="text/html;charset=utf-8" />
 <html>
   <head>
     <title>Hello W\xc3\xb6rld!</title>
   </head>
   <body>Hi all</body>
 </html>'''

In [352]:
import lxml.html

In [353]:
def processRecord(tag_soup):
    #import lxml.html.usedoctest
    #page = urlopen(url)
    #root = lxml.html.fromstring(tag_soup)
    soup = BeautifulSoup(tag_soup, "html.parser")
    #body = fromstring(tag_soup).find('.//body')
    #rtnVal = soup.prettify()
    print (list(soup.children))
    return soup.find_all('head')


In [345]:
tag_soup = '''<meta/><head><title>Hello</head><body onload=crash()>Hi all<p>'''

In [354]:
urls_lines =  sc.parallelize( ([tag_soup]))

In [355]:
urls_lines.collect()

['   <meta http-equiv="Content-Type"\n       content="text/html;charset=utf-8" />\n <html>\n   <head>\n     <title>Hello WÃ¶rld!</title>\n   </head>\n   <body>Hi all</body>\n </html>']

In [356]:
temp = urls_lines.map(processRecord)

In [357]:
temp_rdd = temp.collect()

In [358]:
for elem in temp_rdd:
    print(elem)

[<head>
<title>Hello WÃ¶rld!</title>
</head>]


In [322]:
from lxml.html.soupparser import fromstring

In [239]:
root = fromstring(tag_soup)

In [240]:
from lxml.etree import tostring

In [241]:
print(tostring(root, pretty_print=True).strip())

b'<html>\n  <meta/>\n  <head>\n    <title>Hello</title>\n  </head>\n  <body onload="crash()">Hi all<p/></body>\n</html>'


In [242]:
tag_soup = '<body>&copy;&euro;&#45;&#245;&#445;<p>'

In [243]:
body = fromstring(tag_soup).find('.//body')

In [244]:
body.text

'©€-õƽ'

In [245]:
tostring(body)

b'<body>&#169;&#8364;-&#245;&#445;<p/></body>'

In [246]:
tostring(body, method="html")

b'<body>&#169;&#8364;-&#245;&#445;<p></p></body>'

In [359]:
tag_soup = '''\
   <meta http-equiv="Content-Type"
       content="text/html;charset=utf-8" />
 <html>
   <head>
     <title>Hello W\xc3\xb6rld!</title>
   </head>
   <body>Hi all</body>
 </html>'''

In [360]:
import lxml.html

In [413]:
def processRecord(files):
    #import lxml.html.usedoctest
    #page = urlopen(url)
    #root = lxml.html.fromstring(tag_soup)
    soup_l = [BeautifulSoup(file, "html.parser") for file in files ]
    #body = fromstring(tag_soup).find('.//body')
    #rtnVal = soup.prettify()
    #print (list(soup.children))
    
    return [soup.find_all('p') for soup in soup_l]


In [362]:
urls_lines =  sc.parallelize( ([tag_soup]))

In [371]:
urls_lines.show()

AttributeError: 'RDD' object has no attribute 'show'

In [364]:
temp = urls_lines.map(processRecord)

In [365]:
temp_rdd = temp.collect()

In [397]:
for elem in temp_rdd:
    print(elem)

[<head>
<title>Hello WÃ¶rld!</title>
</head>]


In [377]:
#rdd_l = spark.sparkContext.wholeTextFiles("./*.html")

In [398]:
rdd_l=sc.textFile("./*.html")

In [400]:
rdd_l.collect()

11351

In [414]:
temp2 = rdd_l.map(processRecord)

In [415]:
temp_rdd2 = temp2.collect()

In [ ]:
for elem in temp_rdd2:
    print(elem)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],